In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

# Read CSV
df = pd.read_csv(r'F:\ITI\Big Data\Project\mimic-iii-clinical-database-demo-1.4\ICUSTAYS.csv')

# Convert intime and outtime to Hive-compatible datetime strings
df['intime'] = pd.to_datetime(df['intime'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
df['outtime'] = pd.to_datetime(df['outtime'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

# Fill missing intime/outtime with sentinel
df['intime'] = df['intime'].fillna('9999-12-31 00:00:00')
df['outtime'] = df['outtime'].fillna('9999-12-31 00:00:00')

# Fill missing los (length of stay) with -1
df['los'] = pd.to_numeric(df['los'], errors='coerce').fillna(-1)

# Fill missing ward IDs with -1 and convert to nullable integer
df['first_wardid'] = pd.to_numeric(df['first_wardid'], errors='coerce').fillna(-1).astype('Int64')
df['last_wardid'] = pd.to_numeric(df['last_wardid'], errors='coerce').fillna(-1).astype('Int64')

# Categorical columns
cat_cols = ['dbsource', 'first_careunit', 'last_careunit']
for col in cat_cols:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.add_categories('Unknown')
    df[col] = df[col].fillna('Unknown')

# Show any remaining nulls
print("Remaining NaN values:")
print(df.isna().sum())

# Convert to Arrow Table
table = pa.Table.from_pandas(df)

# Save to Parquet in the same folder
pq.write_table(table, r'F:\ITI\Big Data\Project\mimic-iii-clinical-database-demo-1.4\ICUSTAYS.parquet')

# Print schema for verification
print(table.schema)

Remaining NaN values:
row_id            0
subject_id        0
hadm_id           0
icustay_id        0
dbsource          0
first_careunit    0
last_careunit     0
first_wardid      0
last_wardid       0
intime            0
outtime           0
los               0
dtype: int64
row_id: int64
subject_id: int64
hadm_id: int64
icustay_id: int64
dbsource: dictionary<values=string, indices=int8, ordered=0>
first_careunit: dictionary<values=string, indices=int8, ordered=0>
last_careunit: dictionary<values=string, indices=int8, ordered=0>
first_wardid: int64
last_wardid: int64
intime: string
outtime: string
los: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1769
